# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

training_file = "data/traffic-signs-data/train.p"
validation_file= "data/traffic-signs-data/valid.p"
testing_file = "data/traffic-signs-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

import numpy as np

n_train = len(X_train)

n_validation = len(X_valid)

n_test = len(X_test)

image_shape = X_train[0].shape

n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import random
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

def plot_histogram(data, n_classes, title, color='red', y_limit=None):
    mindat = min(data)
    maxdat = max(data)
    bins = np.linspace(mindat, maxdat, n_classes)
    plt.xlim([mindat, maxdat])
    totals, _, _ = plt.hist(data, bins=bins, color=color, alpha=.5, edgecolor='black')
    plt.title(title)
    plt.xlabel('Label (Class)')
    plt.ylabel('Number of Examples')
    plt.show()
    return totals


train_histogram = plot_histogram(y_train, n_classes, 'Training Data', color='green')
plot_histogram(y_valid, n_classes, 'Validation Data', color='blue')
plot_histogram(y_test, n_classes, 'Testing Data')

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
import cv2
import random
import os

In [ ]:
def jitter(img, translation=2, rotation=15, scaling=0.1):
    width, height = img.shape[:2]
    # Translation
    tx = random.randint(-translation, translation)
    ty = random.randint(-translation, translation)
    M = np.float32([[1,0,tx], [0,1,ty]])
    img = cv2.warpAffine(img, M, (height, width))
    # Rotation
    angle = random.randint(-rotation, rotation)
    M = cv2.getRotationMatrix2D((height/2, width/2), angle, 1)
    img = cv2.warpAffine(img, M, (height, width))
    # Scale
    scale = random.uniform(1. - scaling, 1. + scaling)
    _width, _height = int(scale * width), int(scale * height)
    img = cv2.resize(img,(_height, _width), interpolation = cv2.INTER_CUBIC)
    left, top = int((_width - width) / 2), int((_height - height) / 2)
    right, bottom = _width - left - width, _height - top - height
    if left < 0 or right < 0:  # down-scale, add borders
        img = cv2.copyMakeBorder(img, top=-top, bottom=-bottom, left=-left, right=-right, \
                                 borderType= cv2.BORDER_CONSTANT, value=[0,0,0])
    else: # up-scale, crop
        img = img[left:left+width, top:top+height]
    
    assert ((width, height) == img.shape[:2])
    return img


def enrich_data(x, y, histogram, extra=0.1):
    classes = {}
    for i in range(len(x)):
        if y[i] not in classes:
            classes[y[i]] = []
        classes[y[i]].append(x[i])
    
    n_max = int(max(histogram))
    n_max += int(n_max * extra)
    for label in range(len(histogram)):
        print('Enriching class:', label)
        n = len(classes[label])
        for _ in range(n, n_max+1):
            i = random.randint(0, n-1)
            new_img = jitter(classes[label][i])
            x = np.append(x, [new_img], axis=0)
            y = np.append(y, [label], axis=0)
        print('Adding', (n_max-n), 'new training data to class', label)
            
    return x, y


enriched_training_file = '{}.enriched'.format(training_file)
if os.path.exists(enriched_training_file):
    print('Enriched training file is exist, using it.')
    with open(enriched_training_file, mode='rb') as f:
        train = pickle.load(f)
        X_train, y_train = train['features'], train['labels']
else:
    X_train, y_train = enrich_data(X_train, y_train, train_histogram)
    with open(enriched_training_file, mode='wb') as f:
        print('Saving enriched training data into file', enriched_training_file)
        train['features'] = X_train
        train['labels'] = y_train
        pickle.dump(train, f)

plot_histogram(y_train, n_classes, 'Enriched Training Data', color='green')

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

def preprocess(x):
    for i in range(len(x)):
        img = x[i]
        # Histogram equalization (http://docs.opencv.org/3.1.0/d5/daf/tutorial_py_histogram_equalization.html)
        hist,bins = np.histogram(img.flatten(), 256, [0, 256])
        cdf = hist.cumsum()
        cdf_normalized = cdf * hist.max() / cdf.max()
        cdf_m = np.ma.masked_equal(cdf, 0.)
        cdf_m = (cdf_m - cdf_m.min()) * 255. / (cdf_m.max() - cdf_m.min())
        cdf = np.ma.filled(cdf_m, 0).astype('uint8')
        img = cdf[img]
        # Convert to YUV
        img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        x[i] = img
        
    x = x.astype(np.float)
    x = (x - 128.) / 128.
    
    return x

preprocess(X_train)
print('Preprocessed training data')

preprocess(X_valid)
print('Preprocessed validation data')

preprocess(X_test)
print('Preprocessed testing data')

In [ ]:
from sklearn.utils import shuffle
import random

X_train, y_train = shuffle(X_train, y_train)

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

from tensorflow.contrib.layers import flatten
import tensorflow as tf
import timeit
import os

In [ ]:
class TrafficSignNet:
    def __init__(self, width=32, height=32, depth=3, n_classes=43, n_features1=108, n_features2=200, \
                 n_hidden1=100, n_hidden2=50, mu=0, sigma=0.1, learning_rate=0.001, epsilon=0.001):
        self.depth = depth
        self.n_classes = n_classes
        self.mu = mu
        self.sigma = sigma
        self.n_features1 = n_features1
        self.n_features2 = n_features2
        self.n_hidden1 = n_hidden1
        self.n_hidden2 = n_hidden2
        self.learning_rate = learning_rate
        self.epsilon = epsilon

        # Input/output layers
        self.x = tf.placeholder(tf.float32, (None, width, height, depth), name='x')
        self.y = tf.placeholder(tf.int32, (None), name='y')
        self.one_hot_y = tf.one_hot(self.y, n_classes)
        
        # batch normalization parameter
        self.is_training = tf.placeholder(tf.bool, name='is_training')
        
        self._init_convolution_layer1()
        self._init_convolution_layer2()
        self._init_combine_convolution_layers()
        self._init_fully_connected_layer()
        self._init_logit_layer()
        self._init_operations()
    
    
    def _init_convolution_layer1(self):
        # Convolution layer 1
        self.conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, self.depth, self.n_features1), \
                                                       mean=self.mu, stddev=self.sigma), \
                                   name='conv1_W')
        self.conv1_b = tf.Variable(tf.zeros(self.n_features1), name='conv1_b')
        conv1 = tf.nn.conv2d(self.x, self.conv1_W, strides=[1, 1, 1, 1], padding='SAME') + self.conv1_b
        conv1 = tf.nn.relu(conv1)
        # Pooling 1, input=32x32x108, output=8x8x108
        self.conv1 = tf.nn.max_pool(conv1, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='VALID')
        
        
    def _init_convolution_layer2(self):
        # Convolution Layer 2, convolution, input=8x8x108, output=8x8x200
        self.conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, self.n_features1, self.n_features2), \
                                                       mean=self.mu, stddev=self.sigma), \
                                   name='conv2_W')
        self.conv2_b = tf.Variable(tf.zeros(self.n_features2), name='conv2_b')
        conv2 = tf.nn.conv2d(self.conv1, self.conv2_W, strides=[1, 1, 1, 1], padding='SAME') + self.conv2_b
        conv2 = tf.nn.relu(conv2)
        # Pooling 2, input=8x8x200, output=2x2x200
        self.conv2 = tf.nn.max_pool(conv2, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='VALID')
        
        
    def _init_combine_convolution_layers(self):
        # Flatten 1
        self.fc0_1 = flatten(self.conv1)
        # Flatten 2
        self.fc0_2 = flatten(self.conv2)
        # Concat 1 and 2
        self.fc0 = tf.concat([self.fc0_1, self.fc0_2], 1)

        
    def _init_bn_fully_connected_layer(self):
        size0 = int(self.fc0.shape[1])
        # layer 1
        fc1 = tf.contrib.layers.fully_connected(self.fc0, self.n_hidden1, activation_fn=None)
        fc1 = tf.contrib.layers.batch_norm(fc1, center=True, scale=True, is_training=self.is_training)
        self.fc1 = tf.nn.relu(fc1)
        # layer 2
        fc2 = tf.contrib.layers.fully_connected(self.fc1, self.n_hidden2, activation_fn=None)
        fc2 = tf.contrib.layers.batch_norm(fc2, center=True, scale=True, is_training=self.is_training)
        self.fc2 = tf.nn.relu(fc2)
        
        
    def _init_fully_connected_layer(self):
        # Fully connected layer 1
        size0 = int(self.fc0.shape[1])
        self.fc1_W = tf.Variable(tf.truncated_normal(shape=(size0, self.n_hidden1), \
                                                     mean=self.mu, stddev=self.sigma), \
                                 name='fc1_W')
        self.fc1_b = tf.Variable(tf.zeros(self.n_hidden1), name='fc1_b')
        fc1 = tf.matmul(self.fc0, self.fc1_W) + self.fc1_b
        self.fc1 = tf.nn.relu(fc1)
        # Fully connected layer 2
        self.fc2_W = tf.Variable(tf.truncated_normal(shape=(self.n_hidden1, self.n_hidden2), \
                                                     mean=self.mu, stddev=self.sigma), \
                                 name='fc2_W')
        self.fc2_b = tf.Variable(tf.zeros(self.n_hidden2), name='fc2_b')
        fc2 = tf.matmul(self.fc1, self.fc2_W) + self.fc2_b
        self.fc2 = tf.nn.relu(fc2)
        
        
    def _init_logit_layer(self):
        # Logit Layer
        self.fc3_W = tf.Variable(tf.truncated_normal(shape=(self.n_hidden2, self.n_classes), \
                                                     mean=self.mu, stddev=self.sigma), \
                                 name='fc3_W')
        self.fc3_b = tf.Variable(tf.zeros(self.n_classes), name='fc3_b')
        self.logits = tf.matmul(self.fc2, self.fc3_W) + self.fc3_b
    
    
    def _init_operations(self):
        # Loss operation
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.one_hot_y, logits=self.logits)
        self.loss_operation = tf.reduce_mean(self.cross_entropy)
        #self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
        self.optimizer = tf.train.AdagradOptimizer(learning_rate=self.learning_rate)
        self.train_operation = self.optimizer.minimize(self.loss_operation)

        self.correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.one_hot_y, 1))
        self.accuracy_operation = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
    
    
    def evaluate(self, X_data, y_data, session, batch_size=128):
        """Evaluate the data and then measure the model's accuracy"""
        num_data = len(X_data)
        total_accuracy = 0
        for offset in range(0, num_data, batch_size):
            end = offset + batch_size
            batch_x, batch_y = X_data[offset:end], y_data[offset:end]
            accuracy = session.run(self.accuracy_operation,
                                   feed_dict={self.x: batch_x, self.y: batch_y, self.is_training: False})
            total_accuracy += accuracy * len(batch_x)
        return total_accuracy / num_data

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

import os


epochs = 200
batch_size = 128
learning_rate = 0.001

net = TrafficSignNet(n_classes=n_classes, learning_rate=learning_rate)

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_data = len(X_train)    
    start_time = timeit.default_timer()
    
    print('TRAINING')
    for i in range(epochs):
        X_train, y_train = shuffle(X_train, y_train)
        train_accuracy = 0.
        for offset in range(0, num_data, batch_size):
            batch_x, batch_y = X_train[offset:offset+batch_size], y_train[offset:offset+batch_size]
            _, accuracy = sess.run([net.train_operation, net.accuracy_operation],
                                   feed_dict={net.x: batch_x, net.y: batch_y})
            train_accuracy += accuracy * len(batch_x)
        train_accuracy /= num_data        
        validation_accuracy = net.evaluate(X_valid, y_valid, sess, batch_size=batch_size)
        time_elapsed = timeit.default_timer() - start_time
        print('EPOCH {} {:.2f}s - Accuracy: training={:.5f}, validation={:.5f}'.format(
                i+1, time_elapsed, train_accuracy, validation_accuracy))
        if (i+1) % 100 == 0:
            saver.save(sess, 'models/model-{}'.format(int(start_time)), global_step=(i+1))
    
    saver.save(sess, 'models/model-{}'.format(int(start_time)), global_step=epochs)
    print('Model saved')

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('models/'))

    test_accuracy = net.evaluate(X_test, y_test, sess)
    print("Test Accuracy = {:.5f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")